In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("humansintheloop/dollar-street-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'dollar-street-dataset' dataset.
Path to dataset files: /kaggle/input/dollar-street-dataset


In [2]:
cp -r "/kaggle/input/dollar-street-dataset/Dollar street trial" "/content"


In [3]:
!find "/content/Dollar street trial" -type d -name "Abstract_*" -exec rm -rf {} +


In [4]:
!mkdir -p "/content/dataset/ann"
!mkdir -p "/content/dataset/img"


In [5]:
!find "/content/Dollar street trial" -type f -path "*/ann/*.json" -exec mv {} "/content/dataset/ann/" \;


In [6]:
!find "/content/Dollar street trial" -type f -path "*/img/*" -exec mv {} /content/dataset/img/ \;


In [7]:
import os
import shutil
import random

random.seed(42)

base = "/content/dataset"
img_dir = f"{base}/img"
ann_dir = f"{base}/ann"

# Create YOLO folders
for split in ["train", "val", "test"]:
    os.makedirs(f"{base}/images/{split}", exist_ok=True)
    os.makedirs(f"{base}/labels/{split}", exist_ok=True)

# List all images
images = [f for f in os.listdir(img_dir) if f.endswith(".jpg")]
random.shuffle(images)

# 80/10/10 split
n = len(images)
train_split = int(0.8 * n)
val_split = int(0.1 * n)

train_files = images[:train_split]
val_files = images[train_split:train_split + val_split]
test_files = images[train_split + val_split:]

def move_files(file_list, split):
    for img_file in file_list:
        img_path = os.path.join(img_dir, img_file)

        # JSON annotation must have same filename
        ann_file = img_file+ ".json"
        ann_path = os.path.join(ann_dir, ann_file)

        # Copy images
        shutil.copy(img_path, f"{base}/images/{split}/{img_file}")

        # Copy labels (skip if missing)
        if os.path.exists(ann_path):
            shutil.copy(ann_path, f"{base}/labels/{split}/{ann_file}")

move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

print("Dataset successfully split!")


Dataset successfully split!


In [8]:
!ls -1 "/content/dataset/images/train" | wc -l

18160


In [9]:
!ls -1 "/content/dataset/labels/val" | wc -l

2270


In [10]:
!ls -1 "/content/dataset/labels/test" | wc -l

2270


In [11]:
import os
import json
import yaml  # PyYAML (already installed in Colab)

base_path = "/content/dataset"
ann_dir = f"{base_path}/ann"

# 1. Collect all class names
class_names = set()

for file in os.listdir(ann_dir):
    if file.endswith(".json"):
        with open(os.path.join(ann_dir, file), "r") as f:
            data = json.load(f)

        objects = data.get("objects", [])
        for obj in objects:
            if "classTitle" in obj:
                class_names.add(obj["classTitle"])

class_names = sorted(list(class_names))

print("Found classes:", class_names)

# 2. Build YAML structure
dataset_yaml = {
    "path": base_path,
    "train": "images/train",
    "val": "images/val",
    "test": "images/test",
    "names": {i: name for i, name in enumerate(class_names)}
}

# 3. Save YAML file
save_path = f"{base_path}/dataset.yaml"

with open(save_path, "w") as f:
    yaml.dump(dataset_yaml, f, sort_keys=False)

print(f"dataset.yaml created at: {save_path}")


Found classes: ['Alcoholic drink', 'Armchair', 'Bathroom door', 'Bed', 'Bike', 'Book', 'Bowl', 'Car', 'Car key', 'Chicken', 'Cleaning equipment', 'Cleaning floor', 'Computer', 'Cooking pot', 'Cooking utensils', 'Cup', 'Cutlery', 'Diaper', 'Dish brush', 'Dish rack', 'Dish washing soap', 'Dishwasher', 'Earings', 'Family snapshot', 'Freezers', 'Front door', 'Frontdoor keys', 'Fruit trees', 'Fruits', 'Glasses', 'Goat', 'Grains', 'Guest beds', 'Hair brush', 'Hand', 'Home', 'Instrument', 'Jacket', 'Jewelry', 'Kids bed', 'Kitchen sink', 'Light sources', 'Light sources by bed, reading light', 'Light sources in living room', 'Light sources kitchen', 'Lock on front door', 'Make up', 'Meat', 'Medication', 'Menstruation pads', 'Mosquito protections', 'Motorcycle', 'Music equipment', 'Necklace', 'Oven', 'Palms', 'Paper', 'Pen', 'Pet', 'Pet food', 'Phone', 'Plate', 'Plates of food', 'Power outlet', 'Power switch', 'Radio', 'Refrigerator', 'Roof', 'Rug', 'Salt', 'Shampoo', 'Shoe', 'Shower', 'Soap', '

In [12]:
import yaml

yaml_path = "/content/dataset/dataset.yaml"

with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

# 'names' is a dictionary: {0: 'class1', 1: 'class2', ...}
num_classes = len(data.get("names", {}))
print("Number of classes:", num_classes)


Number of classes: 94


In [13]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.9 MB/s eta 0:00:00


In [14]:
import os
import json

base_path = "/content/dataset"
image_dir = f"{base_path}/images"
label_dir = f"{base_path}/labels"

# Load class mapping from dataset.yaml
import yaml
with open(f"{base_path}/dataset.yaml") as f:
    data_yaml = yaml.safe_load(f)

class_mapping = data_yaml["names"]  # {0: 'Plates of food', 1: 'Chair', ...}
class_name_to_id = {v: k for k, v in class_mapping.items()}

splits = ["train", "val", "test"]

for split in splits:
    img_split_dir = os.path.join(image_dir, split)
    lbl_split_dir = os.path.join(label_dir, split)
    os.makedirs(lbl_split_dir, exist_ok=True)

    for img_file in os.listdir(img_split_dir):
        if not img_file.endswith(".jpg"):
            continue

        img_path = os.path.join(img_split_dir, img_file)
        ann_file = os.path.join(base_path, "ann", img_file + ".json")

        if not os.path.exists(ann_file):
            continue

        with open(ann_file) as f:
            data = json.load(f)

        h = data["size"]["height"]
        w = data["size"]["width"]

        lines = []
        for obj in data.get("objects", []):
            class_name = obj["classTitle"]
            class_id = class_name_to_id[class_name]

            x1, y1 = obj["points"]["exterior"][0]
            x2, y2 = obj["points"]["exterior"][1]

            # YOLO normalized format
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            width = (x2 - x1) / w
            height = (y2 - y1) / h

            lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")

        # Save TXT label
        txt_path = os.path.join(lbl_split_dir, img_file.replace(".jpg", ".txt"))
        with open(txt_path, "w") as f:
            f.write("\n".join(lines))


In [ ]:
from ultralytics import YOLO

# Load the YOLOv8s model
model = YOLO('yolov8s.yaml')

# Train the model
model.train(data='/content/dataset/dataset.yaml', epochs=50, imgsz=640, batch=64)


Ultralytics 8.3.237 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.yaml, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, po